# Applied data science capstone project week3 : part I

##Wikipedia page :  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada: M

In [188]:
# libraries
import numpy as np
import requests
import lxml.html as lh
import pandas as pd
print('Done')

Done


In [0]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# store in a page
page = requests.get(url)
#Store content
doc = lh.fromstring(page.content)
#get the data stored between <tr>..</tr> 
tr_elements = doc.xpath('//tr')

### To check columns

In [190]:
#Check the length of the first 10 rows
[len(T) for T in tr_elements[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

### To get the headers

In [191]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


### Creating a dataframe

In [0]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [193]:
[len(C) for (title,C) in col]

[181, 181, 181]

### Now let's create pandas dataframe

In [0]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [195]:
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [196]:
df.shape

(181, 3)

**Renaming the columns**

In [197]:
df.columns = ['Postcode','Borough' ,'Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [198]:
df.dtypes

Postcode         object
Borough          object
Neighbourhood    object
dtype: object

Removing the useless strings

In [199]:
df = df.replace('\n','', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Now drop borough non assigned cells**

In [200]:
df=df.replace("Not assigned",np.nan)
df=df.dropna()
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [201]:
df = df.reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Grouping by Postcode and sub-grouping by Borough

In [202]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [203]:
df.drop(df.index[0], inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Assigning Borough values to the Neighbourhood values where neighbour values are not given

In [206]:
df.shape

(103, 3)

Saving the file to CSV

In [210]:
df.to_csv(r'df_Tor.csv')
print('Saved')

Saved
